# 🌾 CSIRO Image2Biomass Prediction - Complete Solution

**Author:** Manish Kumar Singh  
**Competition:** [CSIRO - Image2Biomass Prediction](https://www.kaggle.com/competitions/csiro-biomass)  
**Objective:** Predict pasture biomass from drone and ground images using deep learning.

---

## 📋 Table of Contents
1. Introduction & Competition Overview
2. Imports & Environment Setup
3. Data Loading & Exploration
4. Image Visualization
5. Data Preprocessing & Augmentation
6. Weighted R² Metric Implementation
7. EfficientNetB0 Model Architecture
8. Training Configuration & Callbacks
9. Model Training
10. Training History Visualization
11. Validation Evaluation
12. Test Predictions & Submission
13. Predictions vs Ground Truth Plots
14. Final Summary & Results

---

# 1️⃣ Introduction

The **CSIRO Image2Biomass Prediction** competition challenges participants to estimate pasture biomass using drone and ground imagery.  
Accurate predictions help improve **farm efficiency**, **animal welfare**, and **soil sustainability**.

### 🎯 Evaluation Metric: Weighted R²
The competition uses a weighted version of the R² (coefficient of determination):

$$
R^2 = 1 - \frac{\sum w_i(y_i - \hat{y}_i)^2}{\sum w_i(y_i - \bar{y})^2}
$$

### 📊 Target Variables & Weights:

| Target | Weight |
|---------|--------|
| Dry_Green_g | 0.1 |
| Dry_Dead_g | 0.1 |
| Dry_Clover_g | 0.1 |
| GDM_g | 0.2 |
| Dry_Total_g | 0.5 |

---

# 2️⃣ 📦 Imports & Environment Setup

In [ ]:
# Step 1: Import all necessary libraries
import os
import cv2
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import backend as K

# Step 2: Configure warnings and plot style
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Step 3: Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Step 4: Display library versions
print(f"✅ TensorFlow: {tf.__version__}")
print(f"✅ NumPy: {np.__version__}")
print(f"✅ Pandas: {pd.__version__}")
print(f"✅ GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

---

# 3️⃣ 📊 Data Loading & Exploration

In [ ]:
# Step 1: Define data path
DATA_PATH = '/kaggle/input/csiro-biomass'

# Step 2: Load CSV files
train_df = pd.read_csv(f"{DATA_PATH}/train.csv")
test_df = pd.read_csv(f"{DATA_PATH}/test.csv")
sample_submission = pd.read_csv(f"{DATA_PATH}/sample_submission.csv")

# Step 3: Define target columns and their weights
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
TARGET_WEIGHTS = [0.1, 0.1, 0.1, 0.2, 0.5]

# Step 4: Display data information
print("="*60)
print("📁 DATASET INFORMATION")
print("="*60)
print(f"\n📊 Train shape: {train_df.shape}")
print(f"📊 Test shape: {test_df.shape}")
print(f"📊 Sample submission shape: {sample_submission.shape}")

print(f"\n🎯 Target columns: {TARGET_COLS}")
print(f"⚖️  Target weights: {TARGET_WEIGHTS}")

print("\n📋 First few rows of training data:")
display(train_df.head())

print("\n📊 Missing values in target columns:")
print(train_df[TARGET_COLS].isnull().sum())

print("\n📊 Target statistics:")
display(train_df[TARGET_COLS].describe())

---

# 4️⃣ 🖼️ Image Visualization

In [ ]:
# Step 1: Define function to visualize sample images
def visualize_samples(df, img_dir, num=6, show_targets=True):
    """
    Visualize random sample images from the dataset.
    
    Args:
        df: DataFrame containing image information
        img_dir: Directory containing images
        num: Number of images to display
        show_targets: Whether to display target values
    """
    # Sample random images
    sample = df.sample(min(num, len(df)), random_state=SEED)
    
    # Create subplot grid
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, (idx, row) in enumerate(sample.iterrows()):
        if i >= num:
            break
            
        # Try different image extensions
        img_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.JPG']:
            path = os.path.join(img_dir, f"{row['image_id']}{ext}")
            if os.path.exists(path):
                img_path = path
                break
        
        # Display image if found
        if img_path and os.path.exists(img_path):
            img = mpimg.imread(img_path)
            axes[i].imshow(img)
            
            # Create title with target information
            if show_targets and 'Dry_Total_g' in df.columns:
                title = f"{row['image_id']}\nTotal Biomass: {row['Dry_Total_g']:.1f}g"
            else:
                title = f"{row['image_id']}"
            
            axes[i].set_title(title, fontsize=10, fontweight='bold')
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, f"Image not found:\n{row['image_id']}", 
                        ha='center', va='center')
            axes[i].axis('off')
    
    plt.suptitle("🌾 Sample Training Images", fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()

# Step 2: Visualize training samples
print("\n📸 Visualizing sample training images...\n")
visualize_samples(train_df, f"{DATA_PATH}/train_images", num=6)

---

# 5️⃣ ⚙️ Data Preprocessing & Augmentation

In [ ]:
# Step 1: Define preprocessing parameters
IMG_SIZE = 224
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# Step 2: Split data into training and validation sets
train_data, val_data = train_test_split(
    train_df, 
    test_size=VAL_SPLIT, 
    random_state=SEED,
    shuffle=True
)

print("="*60)
print("🔄 DATA PREPROCESSING")
print("="*60)
print(f"\n📊 Training samples: {len(train_data)}")
print(f"📊 Validation samples: {len(val_data)}")
print(f"📊 Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"📊 Batch size: {BATCH_SIZE}")

# Step 3: Prepare dataframes with image filenames
train_data_prep = train_data.copy()
val_data_prep = val_data.copy()

# Add .jpg extension to image_id
train_data_prep['image_filename'] = train_data_prep['image_id'] + '.jpg'
val_data_prep['image_filename'] = val_data_prep['image_id'] + '.jpg'

# Step 4: Create ImageDataGenerator with augmentation for training
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Step 5: Create ImageDataGenerator for validation (only rescaling)
val_gen = ImageDataGenerator(rescale=1./255)

# Step 6: Create data flow from dataframes for multi-output regression
train_flow = train_gen.flow_from_dataframe(
    dataframe=train_data_prep,
    directory=f"{DATA_PATH}/train_images",
    x_col='image_filename',
    y_col=TARGET_COLS,  # Multiple target columns
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='raw',
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True
)

val_flow = val_gen.flow_from_dataframe(
    dataframe=val_data_prep,
    directory=f"{DATA_PATH}/train_images",
    x_col='image_filename',
    y_col=TARGET_COLS,  # Multiple target columns
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='raw',
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=False
)

print("\n✅ Data generators created successfully!")
print(f"✅ Training batches per epoch: {len(train_flow)}")
print(f"✅ Validation batches per epoch: {len(val_flow)}")

---

# 6️⃣ 🎯 Weighted R² Metric Implementation

In [ ]:
# Step 1: Define weighted R² score function
def weighted_r2_score(y_true, y_pred, weights=None):
    """
    Calculate weighted R² score for multi-output regression.
    
    Args:
        y_true: True values (n_samples, n_targets)
        y_pred: Predicted values (n_samples, n_targets)
        weights: Weights for each target (n_targets,)
    
    Returns:
        Weighted R² score
    """
    # Set default weights if not provided
    if weights is None:
        weights = np.ones(y_true.shape[1])
    
    weights = np.array(weights)
    
    # Calculate R² for each target
    r2_scores = []
    for i in range(y_true.shape[1]):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]
        
        # Calculate sum of squared residuals and total sum of squares
        ss_res = np.sum((y_t - y_p) ** 2)
        ss_tot = np.sum((y_t - np.mean(y_t)) ** 2)
        
        # Calculate R² score
        r2 = 1 - (ss_res / (ss_tot + 1e-8))
        r2_scores.append(r2)
    
    # Calculate weighted average
    weighted_r2 = np.sum(np.array(r2_scores) * weights) / np.sum(weights)
    return weighted_r2, r2_scores

# Step 2: Test the metric with dummy data
print("="*60)
print("🎯 WEIGHTED R² METRIC")
print("="*60)
test_true = np.random.rand(100, 5)
test_pred = test_true + np.random.rand(100, 5) * 0.1
test_score, test_individual = weighted_r2_score(test_true, test_pred, TARGET_WEIGHTS)
print(f"\n✅ Metric test successful!")
print(f"\n📊 Test weighted R² score: {test_score:.4f}")
print(f"\n📊 Individual R² scores: {[f'{s:.4f}' for s in test_individual]}")

---

# 7️⃣ 🧠 EfficientNetB0 Model Architecture

In [ ]:
# Step 1: Define model building function
def build_model(input_shape=(224, 224, 3), num_outputs=5):
    """
    Build EfficientNetB0-based multi-output regression model.
    
    Args:
        input_shape: Input image shape
        num_outputs: Number of target variables to predict
    
    Returns:
        Compiled Keras model
    """
    # Load pre-trained EfficientNetB0 without top layers
    base = EfficientNetB0(
        include_top=False,
        input_shape=input_shape,
        weights='imagenet'
    )
    
    # Freeze base model for transfer learning
    base.trainable = False
    
    # Build custom top layers
    inputs = layers.Input(shape=input_shape, name='input')
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.Dropout(0.3, name='dropout_1')(x)
    x = layers.Dense(256, activation='relu', name='dense_1')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.Dropout(0.3, name='dropout_2')(x)
    x = layers.Dense(128, activation='relu', name='dense_2')(x)
    x = layers.Dropout(0.2, name='dropout_3')(x)
    
    # Output layer for multiple targets
    outputs = layers.Dense(num_outputs, activation='linear', name='output')(x)
    
    # Create model
    model = models.Model(inputs=inputs, outputs=outputs, name='EfficientNetB0_Biomass')
    
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    return model

# Step 2: Build and display model
print("="*60)
print("🧠 MODEL ARCHITECTURE")
print("="*60)
model = build_model(num_outputs=len(TARGET_COLS))
print(f"\n✅ Model built successfully!")
print(f"\n📊 Total parameters: {model.count_params():,}")
print(f"📊 Output targets: {len(TARGET_COLS)} ({', '.join(TARGET_COLS)})")
print("\n📋 Model Summary:")
model.summary()

---

# 8️⃣ 🏋️ Training Configuration & Callbacks

In [ ]:
# Step 1: Define training parameters
EPOCHS = 30

# Step 2: Define callbacks for training
callbacks_list = [
    # Early stopping to prevent overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1,
        mode='min'
    ),
    
    # Reduce learning rate when plateau is reached
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1,
        mode='min'
    ),
    
    # Save best model
    ModelCheckpoint(
        'best_biomass_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1,
        mode='min'
    )
]

print("="*60)
print("🏋️ TRAINING CONFIGURATION")
print("="*60)
print(f"\n📊 Maximum epochs: {EPOCHS}")
print(f"📊 Batch size: {BATCH_SIZE}")
print(f"📊 Learning rate: 1e-3")
print(f"\n✅ Callbacks configured:")
print(f"  - EarlyStopping (patience=7)")
print(f"  - ReduceLROnPlateau (patience=3, factor=0.5)")
print(f"  - ModelCheckpoint (save best model)")

---

# 9️⃣ 🚀 Model Training

In [ ]:
# Step 1: Start training
print("="*60)
print("🚀 STARTING MODEL TRAINING")
print("="*60)
print(f"\n⏰ Training started...\n")

# Step 2: Fit the model
history = model.fit(
    train_flow,
    validation_data=val_flow,
    epochs=EPOCHS,
    callbacks=callbacks_list,
    verbose=1
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETED!")
print("="*60)
print(f"\n📊 Epochs trained: {len(history.history['loss'])}")
print(f"📊 Best validation loss: {min(history.history['val_loss']):.4f}")
print(f"📊 Best validation MAE: {min(history.history['val_mae']):.4f}")

---

# 🔟 📈 Training History Visualization

In [ ]:
# Step 1: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2, color='#2E86AB')
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2, color='#A23B72')
axes[0].set_title('📉 Model Loss (MSE)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot MAE
axes[1].plot(history.history['mae'], label='Train MAE', linewidth=2, color='#2E86AB')
axes[1].plot(history.history['val_mae'], label='Val MAE', linewidth=2, color='#A23B72')
axes[1].set_title('📊 Model MAE', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('MAE', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Training Summary:")
print(f"  Final Train Loss: {history.history['loss'][-1]:.4f}")
print(f"  Final Val Loss: {history.history['val_loss'][-1]:.4f}")
print(f"  Final Train MAE: {history.history['mae'][-1]:.4f}")
print(f"  Final Val MAE: {history.history['val_mae'][-1]:.4f}")

---

# 1️⃣1️⃣ 🔍 Validation Evaluation

In [ ]:
# Step 1: Generate predictions on validation set
print("="*60)
print("🔍 VALIDATION EVALUATION")
print("="*60)
print(f"\n⏳ Generating predictions on validation set...\n")

# Create validation generator (no shuffle)
val_flow_eval = val_gen.flow_from_dataframe(
    dataframe=val_data_prep,
    directory=f"{DATA_PATH}/train_images",
    x_col='image_filename',
    y_col=TARGET_COLS,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='raw',
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Step 2: Get predictions
val_predictions = model.predict(val_flow_eval, verbose=1)
val_true = val_data_prep[TARGET_COLS].values

# Step 3: Calculate weighted R² score
weighted_r2, individual_r2 = weighted_r2_score(val_true, val_predictions, TARGET_WEIGHTS)

print("\n" + "="*60)
print("📊 VALIDATION RESULTS")
print("="*60)
print(f"\n🎯 Weighted R² Score: {weighted_r2:.4f}")
print("\n📊 Individual Target Performance:\n")
print(f"{'Target':<20} {'R² Score':<12} {'MAE':<12} {'Weight':<10}")
print("-" * 60)
for i, (col, weight) in enumerate(zip(TARGET_COLS, TARGET_WEIGHTS)):
    r2 = individual_r2[i]
    mae = mean_absolute_error(val_true[:, i], val_predictions[:, i])
    print(f"{col:<20} {r2:<12.4f} {mae:<12.2f} {weight:<10}")

---

# 1️⃣2️⃣ 🔮 Test Predictions & Submission

In [ ]:
# Step 1: Prepare test data
print("="*60)
print("🔮 GENERATING TEST PREDICTIONS")
print("="*60)
print(f"\n⏳ Preparing test data...\n")

test_df_prep = test_df.copy()
test_df_prep['image_filename'] = test_df_prep['image_id'] + '.jpg'

# Step 2: Create test data generator
test_gen = ImageDataGenerator(rescale=1./255)
test_flow = test_gen.flow_from_dataframe(
    dataframe=test_df_prep,
    directory=f"{DATA_PATH}/test_images",
    x_col='image_filename',
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Step 3: Generate predictions
print("⏳ Generating predictions...\n")
test_predictions = model.predict(test_flow, verbose=1)

# Step 4: Create submission dataframe
print("\n⏳ Creating submission file...")
submission = sample_submission.copy()

# Map predictions to submission format
for i, col in enumerate(TARGET_COLS):
    # Find rows in submission that match this target
    target_mask = submission['sample_id'].str.contains(col)
    
    # Get image IDs for this target
    image_ids = submission[target_mask]['sample_id'].str.split('_').str[0]
    
    # Create mapping from image_id to index
    image_id_to_idx = {img_id: idx for idx, img_id in enumerate(test_df['image_id'])}
    
    # Map predictions to submission
    pred_values = [test_predictions[image_id_to_idx[img_id], i] for img_id in image_ids]
    submission.loc[target_mask, 'target'] = pred_values

# Step 5: Save submission
submission.to_csv('submission.csv', index=False)

print("\n" + "="*60)
print("✅ SUBMISSION FILE CREATED")
print("="*60)
print(f"\n📄 File: submission.csv")
print(f"📊 Shape: {submission.shape}")
print(f"\n📋 Preview:")
display(submission.head(10))
print(f"\n📊 Prediction Statistics:")
display(submission['target'].describe())

---

# 1️⃣3️⃣ 📊 Predictions vs Ground Truth Visualization

In [ ]:
# Step 1: Create scatter plots for each target
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']

for i, (col, weight, color) in enumerate(zip(TARGET_COLS, TARGET_WEIGHTS, colors)):
    ax = axes[i]
    
    # Scatter plot
    ax.scatter(val_true[:, i], val_predictions[:, i], alpha=0.6, s=40, color=color, edgecolors='black', linewidth=0.5)
    
    # Perfect prediction line
    min_val = min(val_true[:, i].min(), val_predictions[:, i].min())
    max_val = max(val_true[:, i].max(), val_predictions[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2.5, label='Perfect Prediction', alpha=0.8)
    
    # Calculate metrics
    r2 = individual_r2[i]
    mae = mean_absolute_error(val_true[:, i], val_predictions[:, i])
    
    # Labels and title
    ax.set_xlabel('True Values (g)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Predicted Values (g)', fontsize=12, fontweight='bold')
    ax.set_title(f'{col}\nR² = {r2:.4f} | MAE = {mae:.2f}g | Weight = {weight}', 
                fontsize=13, fontweight='bold', pad=10)
    ax.legend(fontsize=10, loc='upper left')
    ax.grid(True, alpha=0.3, linestyle='--')

# Remove extra subplot
fig.delaxes(axes[5])

plt.suptitle('🌾 Validation Set: Predictions vs Ground Truth', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

---

# 1️⃣4️⃣ 🎉 Final Summary & Results

In [ ]:
# Step 1: Display final summary
print("\n" + "="*70)
print(" "*15 + "🌾 CSIRO IMAGE2BIOMASS PREDICTION - FINAL RESULTS")
print("="*70)

print(f"\n📊 MODEL INFORMATION:")
print(f"  • Architecture: EfficientNetB0 (Transfer Learning)")
print(f"  • Total Parameters: {model.count_params():,}")
print(f"  • Input Size: {IMG_SIZE}x{IMG_SIZE}x3")
print(f"  • Output Targets: {len(TARGET_COLS)}")

print(f"\n📊 DATASET INFORMATION:")
print(f"  • Training Samples: {len(train_data)}")
print(f"  • Validation Samples: {len(val_data)}")
print(f"  • Test Samples: {len(test_df)}")
print(f"  • Batch Size: {BATCH_SIZE}")

print(f"\n📊 TRAINING INFORMATION:")
print(f"  • Epochs Trained: {len(history.history['loss'])}")
print(f"  • Best Val Loss: {min(history.history['val_loss']):.4f}")
print(f"  • Best Val MAE: {min(history.history['val_mae']):.4f}")

print(f"\n🎯 VALIDATION PERFORMANCE:")
print(f"  • Weighted R² Score: {weighted_r2:.4f}")
print(f"\n  📊 Individual Target Performance:")
print(f"  {'-'*66}")
print(f"  {'Target':<20} {'R²':<12} {'MAE (g)':<15} {'Weight':<10}")
print(f"  {'-'*66}")
for i, (col, weight) in enumerate(zip(TARGET_COLS, TARGET_WEIGHTS)):
    r2 = individual_r2[i]
    mae = mean_absolute_error(val_true[:, i], val_predictions[:, i])
    print(f"  {col:<20} {r2:<12.4f} {mae:<15.2f} {weight:<10}")

print(f"\n📄 SUBMISSION:")
print(f"  • File: submission.csv")
print(f"  • Shape: {submission.shape}")
print(f"  • Status: ✅ Ready for submission")

print(f"\n" + "="*70)
print(" "*20 + "🎉 NOTEBOOK EXECUTION COMPLETE! 🎉")
print("="*70)

print(f"\n💡 Next Steps:")
print(f"  1. Download 'submission.csv'")
print(f"  2. Submit to Kaggle competition")
print(f"  3. Fine-tune hyperparameters for better performance")
print(f"  4. Try unfreezing some EfficientNetB0 layers for fine-tuning")
print(f"  5. Experiment with ensemble methods\n")